In [4]:
import pandas as pd
from pathlib import Path

file_path = Path("clean.xlsx")

if not file_path.exists():
    raise FileNotFoundError(f"{file_path} not found in current directory: {Path.cwd()}")

xls = pd.ExcelFile(file_path)
print("Sheets found:", xls.sheet_names)

for sheet in xls.sheet_names:
    df = xls.parse(sheet_name=sheet, nrows=0)  # read only header row to get columns
    cols = list(df.columns)
    print(f"Sheet: {sheet} — {len(cols)} columns")


Sheets found: ['MajorCountry', 'byAge', 'byEduc', 'bySex', 'byCivStat', 'byOccu', 'all countries', 'REGION']
Sheet: MajorCountry — 12 columns
Sheet: byAge — 43 columns
Sheet: byEduc — 36 columns
Sheet: bySex — 3 columns
Sheet: byCivStat — 7 columns
Sheet: byOccu — 43 columns
Sheet: all countries — 44 columns
Sheet: REGION — 36 columns


In [5]:
import pandas as pd
from pathlib import Path

file_path = Path("2.xlsx")

if not file_path.exists():
    raise FileNotFoundError(f"{file_path} not found in directory: {Path.cwd()}")

xls = pd.ExcelFile(file_path)

# -----------------------------
# Helper to melt + align schema
# -----------------------------
def melt_sheet(df, id_col, value_col_name, additional_col_name=None):
    """Melt a sheet from wide to long"""
    long_df = df.melt(
        id_vars=[id_col],
        var_name="year",
        value_name=value_col_name
    ).rename(columns={id_col: additional_col_name if additional_col_name else "category"})
    
    # Extract numeric year
    long_df["year"] = long_df["year"].astype(str).str.extract(r"(\d{4})")
    long_df["year"] = long_df["year"].astype(float).astype("Int64")
    
    return long_df


# -----------------------------
# 1. ALL COUNTRIES
# -----------------------------
df_country = xls.parse("all countries").drop(columns=["Unnamed: 1"], errors="ignore")
countries_long = melt_sheet(df_country, "COUNTRY", "value", "country")

# -----------------------------
# 2. BY AGE
# -----------------------------
df_age = xls.parse("byAge")
age_long = melt_sheet(df_age, "AGE GROUP", "value", "age_group")

# -----------------------------
# 3. BY EDUCATION
# -----------------------------
df_educ = xls.parse("byEduc")
educ_long = melt_sheet(df_educ, "EDUCATIONAL ATTAINMENT", "value", "education")

# -----------------------------
# 4. BY OCCUPATION
# -----------------------------
df_occu = xls.parse("byOccu")
occu_long = melt_sheet(df_occu, "MAJOR OCCUPATION GROUP", "value", "occupation")

countries_long["source"] = "all_countries"
age_long["source"] = "byAge"
educ_long["source"] = "byEduc"
occu_long["source"] = "byOccu"




In [6]:
age_long

,age_group,year,value,source
0,14 - Below,1981,9588,byAge
1,15 - 19,1981,4712,byAge
2,20 - 24,1981,5846,byAge
3,25 - 29,1981,5919,byAge
4,30 - 34,1981,4048,byAge
...,...,...,...,...
541,50 - 54,2022,1279,byAge
542,55 - 59,2022,1006,byAge
543,60 - 64,2022,786,byAge
544,65 - 69,2022,563,byAge


In [7]:
educ_long

,education,year,value,source
0,Not of Schooling Age,1988,5514.0,byEduc
1,No Formal Education,1988,459.0,byEduc
2,Elementary Level,1988,8847.0,byEduc
3,Elementary Graduate,1988,3012.0,byEduc
4,High School Level,1988,7291.0,byEduc
...,...,...,...,...
485,College Graduate,2022,14906.0,byEduc
486,Post Graduate Level,2022,341.0,byEduc
487,Post Graduate,2022,689.0,byEduc
488,Non-Formal Education,2022,756.0,byEduc


In [8]:
occu_long

,occupation,year,value,source
0,"Prof'l, Tech'l, & Related Workers",1981,4821,byOccu
1,Administrative Workers,1981,451,byOccu
2,Clerical Workers,1981,2475,byOccu
3,Sales Workers,1981,1628,byOccu
4,Service Workers,1981,2381,byOccu
...,...,...,...,...
625,Students,2022,9772.0,byOccu
626,Minors (Below 7 years old),2022,2420.0,byOccu
627,Out of School Youth,2022,163.0,byOccu
628,Refugees,2022,NaN,byOccu


In [9]:
countries_long['value'] = countries_long['value'].fillna(0)
age_long['value'] = age_long['value'].fillna(0)
educ_long['value'] = educ_long['value'].fillna(0)
occu_long['value'] = occu_long['value'].fillna(0)


In [10]:
educ_pivot = educ_long.pivot_table(index='year', columns='education', values='value', aggfunc='sum').reset_index()
age_pivot = age_long.pivot_table(index='year', columns='age_group', values='value', aggfunc='sum').reset_index()
occu_pivot = occu_long.pivot_table(index='year', columns='occupation', values='value', aggfunc='sum').reset_index()
countries_pivot = countries_long.pivot_table(index='year', columns='country', values='value', aggfunc='sum').reset_index()    
sex_pivot = xls.parse("bySex")
civ_pivot = xls.parse("byCivStat")

In [11]:
import pandas as pd

# Function to clean column names
def clean_columns(df):
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
    return df

# Clean each pivot table
educ_pivot.columns = educ_pivot.columns.str.strip().str.lower().str.replace(' ', '_')
occu_pivot.columns = occu_pivot.columns.str.strip().str.lower().str.replace(' ', '_')
countries_pivot.columns = countries_pivot.columns.str.strip().str.lower().str.replace(' ', '_')

# Clean 'bySex' table (already partially cleaned)
sex_pivot.columns = sex_pivot.columns.str.strip().str.lower().str.replace(' ', '_')
# normalize any remaining odd chars in column names
sex_pivot.rename(columns=lambda c: c.replace('.', '').replace('-', '_'), inplace=True)

# Convert year and numeric columns to proper numeric types
sex_pivot['year'] = pd.to_numeric(sex_pivot['year'], errors='coerce').astype('Int64')
for col in sex_pivot.columns:
    if col != 'year':
        sex_pivot[col] = pd.to_numeric(sex_pivot[col].astype(str).str.replace(',', ''), errors='coerce').fillna(0).astype('Int64')

# For 'byCivStat' pivot
civ_pivot = xls.parse("byCivStat")  # Load the sheet
civ_pivot.columns = civ_pivot.columns.str.strip().str.lower().str.replace(' ', '_')  # Clean the columns

# Now you can check the cleaned column names
print(educ_pivot.columns)
print(age_pivot.columns)
print(occu_pivot.columns)
print(countries_pivot.columns)
print(sex_pivot.columns)
print(civ_pivot.columns)


Index(['year', 'college_graduate', 'college_level', 'elementary_graduate',
       'elementary_level', 'high_school_graduate', 'high_school_level',
       'no_formal_education', 'non-formal_education',
       'not_reported_/_no_response', 'not_of_schooling_age', 'post_graduate',
       'post_graduate_level', 'vocational_graduate', 'vocational_level'],
      dtype='object', name='education')
Index(['year', '14 - Below', '15 - 19', '20 - 24', '25 - 29', '30 - 34',
       '35 - 39', '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64',
       '65 - 69', '70 - Above'],
      dtype='object', name='age_group')
Index(['year', 'administrative_workers', 'clerical_workers',
       'equipment_operators,_&_laborers', 'housewives',
       'members_of_the_armed_forces', 'minors_(below_7_years_old)',
       'no_occupation_reported', 'out_of_school_youth',
       'prof'l,_tech'l,_&_related_workers', 'refugees', 'retirees',
       'sales_workers', 'service_workers', 'students', 'workers_&_fishermen'],


In [13]:
# Fill NaNs in long-form tables
for df in (educ_pivot, age_pivot, occu_pivot, countries_pivot):
    df.fillna(0, inplace=True)

# Normalize pivot tables: coerce numeric columns and fill NaNs with 0
def normalize_pivot(piv):
    if 'year' in piv.columns:
        piv['year'] = pd.to_numeric(piv['year'], errors='coerce').astype('Int64')
    for col in piv.columns:
        if col == 'year':
            continue
        # remove thousands separators, coerce to numeric, fill NaNs with 0
        piv[col] = pd.to_numeric(piv[col].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        # if all values are whole numbers, convert to nullable integer dtype
        try:
            if (piv[col] % 1 == 0).all():
                piv[col] = piv[col].astype('Int64')
        except Exception:
            # keep as numeric float if modulo check fails
            pass

for p in (educ_pivot, age_pivot, occu_pivot, countries_pivot, sex_pivot, civ_pivot):
    normalize_pivot(p)

# quick check
print("NaNs filled and dtypes:")
print(educ_pivot.dtypes)
print(age_pivot.dtypes)
print(occu_pivot.dtypes)
print(countries_pivot.dtypes)
print(sex_pivot.dtypes)
print(civ_pivot.dtypes)


NaNs filled and dtypes:
education
year                          Int64
college_graduate              Int64
college_level                 Int64
elementary_graduate           Int64
elementary_level              Int64
high_school_graduate          Int64
high_school_level             Int64
no_formal_education           Int64
non-formal_education          Int64
not_reported_/_no_response    Int64
not_of_schooling_age          Int64
post_graduate                 Int64
post_graduate_level           Int64
vocational_graduate           Int64
vocational_level              Int64
dtype: object
age_group
year          Int64
14 - Below    Int64
15 - 19       Int64
20 - 24       Int64
25 - 29       Int64
30 - 34       Int64
35 - 39       Int64
40 - 44       Int64
45 - 49       Int64
50 - 54       Int64
55 - 59       Int64
60 - 64       Int64
65 - 69       Int64
70 - Above    Int64
dtype: object
occupation
year                                 Int64
administrative_workers               Int64
clerical_wo

In [14]:
# Merge all dataframes by the 'year' column using a left join (you can adjust this based on your needs)
merged_df = educ_pivot

# Merge each of the other dataframes one by one
merged_df = merged_df.merge(age_pivot, on='year', how='outer')  # 'outer' join keeps all years from each table
merged_df = merged_df.merge(occu_pivot, on='year', how='outer')
merged_df = merged_df.merge(countries_pivot, on='year', how='outer')
merged_df = merged_df.merge(sex_pivot, on='year', how='outer')
merged_df = merged_df.merge(civ_pivot, on='year', how='outer')

# Check the merged dataframe


In [15]:
merged_df.fillna(0, inplace=True)
merged_df.to_csv("merged_data.csv", index=False)  # Save to CSV without the index

In [22]:
educ_pivot, age_pivot, occu_pivot, countries_pivot, sex_pivot, civ_pivot

educ_pivot.to_csv("../data/educ_pivot.csv", index=False)  # Save to CSV without the index
age_pivot.to_csv("../data/age_pivot.csv", index=False)
occu_pivot.to_csv("../data/occu_pivot.csv", index=False)
countries_pivot.to_csv("../data/countries_pivot.csv", index=False)
sex_pivot.to_csv("../data/sex_pivot.csv", index=False)
civ_pivot.to_csv("../data/civ_pivot.csv", index=False)